# Data augmentation

This notebook explores data augmentation as a mean to increase the accuracy, particularly on the test set

# Setup

Como a execução é feita no Google Collab, é necessário o carregamento dos dados.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

path = "/content/drive/MyDrive/data.zip"

def unzip_file(path):
  from zipfile import ZipFile
  file_name = path

  with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print('Done')

unzip_file(path)

Done


# Balance Dataset

Listar o número de elementos por classe.

In [ ]:
import os


def files_by_class(folder):
  files = sorted(os.listdir(folder))
  for clas in files:
    print(clas,len(os.listdir(f'{folder}/'+ clas)))

files_by_class('data/train')

00000 211
00001 2221
00002 2251
00003 1411
00004 1981
00005 1861
00006 421
00007 1441
00008 1411
00009 1471
00010 2011
00011 1321
00012 2101
00013 2161
00014 781
00015 631
00016 421
00017 1111
00018 1201
00019 211
00020 361
00021 331
00022 391
00023 511
00024 271
00025 1501
00026 601
00027 241
00028 541
00029 271
00030 451
00031 781
00032 241
00033 690
00034 421
00035 1201
00036 391
00037 211
00038 2071
00039 301
00040 361
00041 241
00042 241


Como verificamos que o dataset não estava balanceado, de modo a tentar reduzir o desbalanceamento.

Vamos calcular a média:

In [ ]:
import os
folder = 'data/train'
classes = os.listdir(folder)
total = 0
for index,c in enumerate(classes):
  length = len(os.listdir(f"{folder}/{c}"))
  classes[index] = (c,length)
  total+=length

print(total/43)

912.8372093023256


In [ ]:
import shutil
import random
from PIL import Image, ImageEnhance
import os

data_path = 'data'
IMAGES_PER_CLASS = 913

Copiar os ficheiros de treino para uma nova pasta.

In [ ]:
shutil.copytree(f'{data_path}/train', f'{data_path}/train_balanced_{IMAGES_PER_CLASS}')

In [ ]:
!pip3 install --user --upgrade pillow

Gerar imagens apartir das imagens originais com pequenas alterações, para as classes onde o número de elementos for menor do que a média.


In [ ]:
classes = os.listdir(f'{data_path}/train_balanced_{IMAGES_PER_CLASS}')
list_img = []
for cla in classes:
    list_img = os.listdir(f'{data_path}/train_balanced_{IMAGES_PER_CLASS}/{cla}')
    random.shuffle(list_img)
    for k in range(len(list_img), IMAGES_PER_CLASS):
        filename = f'{data_path}/train_balanced_{IMAGES_PER_CLASS}/{cla}/{list_img[(k - len(list_img)) % len(list_img)]}'
        if not filename.endswith('png'):
          continue
        im = Image.open(filename)
        r = random.uniform(-10.0,10.0)
        im = im.rotate(r)
        r1 = random.uniform(-3.0,3.0)
        r2 = random.uniform(-3.0,3.0)
        im = im.transform(im.size, Image.Transform.AFFINE, (1, 0, r1, 0, 1, r2))
        r = random.uniform(1.0, 1.3)
        im = ImageEnhance.Sharpness(im.convert('RGB'))
        im = im.enhance(r)
        r = random.uniform(1.0, 1.3)
        im = ImageEnhance.Contrast(im)
        im = im.enhance(r)
        im = im.resize((32,32))
        im.save(f'{data_path}/train_balanced_{IMAGES_PER_CLASS}/{cla}/_{k}.png')


In [ ]:
files_by_class('data/train_balanced_913')

00000 909
00001 2221
00002 2251
00003 1411
00004 1981
00005 1861
00006 912
00007 1441
00008 1411
00009 1471
00010 2011
00011 1321
00012 2101
00013 2161
00014 913
00015 912
00016 912
00017 1111
00018 1201
00019 910
00020 911
00021 911
00022 912
00023 913
00024 911
00025 1501
00026 913
00027 910
00028 913
00029 910
00030 912
00031 913
00032 910
00033 912
00034 912
00035 1201
00036 912
00037 910
00038 2071
00039 911
00040 912
00041 910
00042 910


In [ ]:
!mv data/train data/train_original
!mv data/train_balanced_913 data/train

# Imports and Aux Functions

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import os
import numpy as np

# plotting
import matplotlib.pyplot as plt
import pathlib
from PIL import Image
import IPython.display as display

# to display confusion matrix
import seaborn as sn
import pandas as pd

### Aux functions



In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == classNames

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [32,32])

def get_bytes_and_label(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

def show_data(s1,l1, s2,l2, labels, min):
    fig, ax = plt.subplots()
    X = np.arange(len(s1))

    models = labels
    plt.bar(X, s1, width = 0.4, color = 'b', label=l1)
    plt.bar(X + 0.4, s2, color = 'r', width = 0.4, label = l2)
    plt.xticks(X + 0.4 / 2, models)
    plt.ylim(top = 100, bottom = min)
    plt.legend(loc='upper left')
    plt.show()

def show_batch(image_batch, label_batch):
  columns = 8
  rows = BATCH_SIZE / columns + 1
  plt.figure(figsize=(10, 2 * rows))
  for n in range(BATCH_SIZE):
      ax = plt.subplot(int(rows), columns, n+1)
      plt.imshow((image_batch[n]))
      plt.title(classNames[label_batch[n]==1][0])
      plt.axis('off')


def show_history(history):
    print(history.history.keys())

    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='lower right')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()


def show_accuracies(labels, test, val):

    fig, ax = plt.subplots()
    X = np.arange(len(test))

    plt.bar(X, test, width = 0.4, color = 'b', label='test')
    plt.bar(X + 0.4, val, color = 'r', width = 0.4, label = "val")
    plt.xticks(X + 0.4 / 2, labels)
    plt.ylim(top = 1.0, bottom = 0.97)
    plt.legend(loc='upper left')
    plt.show()



def show_misclassified(predictions, ground_truth, images, num_rows = 5, num_cols=3):

    # Plot the first X test images with wrong predictions.
    num_images = num_rows*num_cols
    print(num_images)
    plt.figure(figsize=(2*2*num_cols, 2*num_rows))
    i = 0
    k = 0
    while k < len(images) and i < num_images:
        predicted_label = np.argmax(predictions[k])
        gt = np.where(ground_truth[k])[0][0]
        if predicted_label != gt:
            plt.subplot(num_rows, 2*num_cols, 2*i+1)
            plot_image(k, predictions[k], gt, images)
            plt.subplot(num_rows, 2*num_cols, 2*i+2)
            plot_value_array(k, predictions[k], ground_truth)
            i += 1
        k += 1
    plt.tight_layout()
    plt.show()


def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array, true_label, img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
      color = 'blue'
    else:
      color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(classNames[predicted_label],
                                100*np.max(predictions_array),
                                classNames[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, true_label[i]
    plt.grid(False)
    plt.xticks(range(8))
    plt.yticks([])
    thisplot = plt.bar(range(8), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[np.where(true_label)[0][0]].set_color('blue')



def show_confusion_matrix(mat, classes):

    df_cm = pd.DataFrame(mat, range(classes), range(classes))
    plt.figure(figsize=(20,20))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='d') # font size

    plt.show()

In [ ]:
def split_train_val(train_offset,dataset):
  dataset_length = dataset.cardinality().numpy()
  print("length:",dataset_length)
  train_size = int(train_offset * dataset_length)
  print("Train size",train_size)
  #val_size = int((1- train_offset) * dataset_length / BATCH_SIZE)
  aux = dataset
  dataset = dataset.take(train_size)
  valset = aux.skip(train_size)
  print("Train",dataset.cardinality())
  print("Val",valset.cardinality())
  return dataset, valset

def print_dataset_len(dataset):
  dataset_length = dataset.cardinality().numpy()
  print("Total images in dataset: ", dataset_length)

## Callbacks

Here we introduce a new callback: ReduceLROnPlateau: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau

When the model is trapped in an area where no improvement can be achieved, it is sometimes benefic to reduce the learning rate.



In [ ]:
def prepare_callbacks(file_path):

    checkpointer = ModelCheckpoint(filepath= file_path,
                               monitor = 'val_accuracy',
                               verbose=1,
                               save_weights_only=True,
                               save_best_only=True)


    earlyStopper = EarlyStopping(monitor='val_loss', min_delta = 0.0001, patience = 10, verbose = 1)

    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000000001, verbose = 1)

    return [checkpointer, earlyStopper, reduceLR]


## Batch size

Batch size is an important parameter when training a network. It can influence speed and generalization, not necessarily in the same direction. There is no golden rule for the batch size but 32 is a commom number to start with.

In here we go to 64 to achieve faster training epochs (rather than 32)

See: https://machinelearningmastery.com/how-to-control-the-speed-and-stability-of-training-neural-networks-with-gradient-descent-batch-size/



In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 32

## Settings

In [ ]:
data_path = 'data'
log_and_model_path = 'log_files'

### Prepare to load images

Tranformação das imagens de .ppm para .png

In [ ]:
#!cp -r data/test_ppm/ data/test/
#!cp -r data/train_ppm/ data/train/
#!cd data/test/; for i in */*.ppm; do convert "$i" "${i%.*}.png"; done
#!cd ../train/; for i in */*.ppm; do convert "$i" "${i%.*}.png"; done
#!cd ../../
#!rm data/test/*/*.ppm
#!rm data/train/*/*.ppm

# Load Images

### Training images

Carregamento das imagens de treino

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_path}/train',
    batch_size = None,
    label_mode = 'categorical',
    image_size=(32,32),
    shuffle=True)
normalize = tf.keras.layers.Rescaling(1.0/255)

print(type(dataset))
dataset = dataset.map(lambda x, y: (normalize(x), y))

Found 52380 files belonging to 43 classes.
<class 'tensorflow.python.data.ops.shuffle_op._ShuffleDataset'>


### Information about image shape and size of training set

In [ ]:
t = next(iter(dataset))
print(t[0].shape, t[1].shape)

# note: this only works if dataset is not repeating
dataset_length = dataset.cardinality().numpy()
print("Total images in dataset: ", dataset_length)

(32, 32, 3) (43,)
Total images in dataset:  52380


Carregar as images de teste.

In [ ]:
testset = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_path}/test',
    image_size = (32, 32),
    batch_size = None,
    label_mode = 'categorical',
    shuffle = True)

test_dataset_length = testset.cardinality().numpy()
testset = testset.map(lambda x, y: (normalize(x), y))
testset = testset.batch(batch_size = BATCH_SIZE)

Found 12630 files belonging to 43 classes.


# Models

In [ ]:
classNames = np.array(os.listdir(f'{data_path}/train'))

### Data augmentation functions

In [ ]:
!pip install tensorflow_addons

In [ ]:
import tensorflow_addons as tfa
import cv2

def process_brightness(image, label):

    img = tf.clip_by_value(tfa.image.random_hsv_in_yiq(image, 0.0, 1.0, 1.0, 0.1, 3.0),0,1)
    return img, label

def process_saturation(image, label):

    img = tf.clip_by_value(tfa.image.random_hsv_in_yiq(image, 0.0, 1.0, 3.0, 1.0, 1.0),0,1)
    return img, label

def process_contrast(image, label):

    img = tf.clip_by_value(tf.image.random_contrast(image, lower=0.1, upper=3.0, seed=None), 0, 1)
    return img, label

def process_hue(image, label):

    img = tf.image.random_hue(image, max_delta=0.2, seed=None)
    return img, label

def process_rotate(image, label):

    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    return img, label

def process_shear(image, label):

    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    sx = tf.random.uniform(shape=(), minval=-0.1, maxval=0.1, dtype=tf.dtypes.float32)
    img = tfa.image.transform(img, [1, sx, -sx*32,   0,1,0,  0,0])
    return img, label

def process_translate(image, label):

    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    tx = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)
    ty = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)
    img = tfa.image.translate(img, [tx,ty])
    return img, label

def process_crop(image, label):

    c = tf.random.uniform(shape=(), minval=24, maxval=32, dtype=tf.dtypes.float32)
    img = tf.image.random_crop(image, size=[c,c,3])
    img = tf.image.resize(img ,size= [32,32])
    return img, label


def process_edges(image, label):
    # Expand dimensions to match expected rank
    expanded_image = tf.expand_dims(image, axis=0)
    # Apply Sobel filter to the expanded image
    sobel = tf.image.sobel_edges(expanded_image)
    # Compute gradient magnitude
    gradient_magnitude = tf.sqrt(tf.square(sobel[..., 0]) + tf.square(sobel[..., 1]))
    # Clip values between 0 and 1
    clipped_img = tf.clip_by_value(gradient_magnitude / 8, 0, 1)
    # Remove the extra dimension
    img = tf.squeeze(clipped_img, axis=0)
    return img, label

def process_fourier(image, label):
    image_float = tf.cast(image, tf.float32)
    image_complex = tf.complex(image_float, tf.zeros_like(image_float))
    dft = tf.signal.fft2d(image_complex)
    shifted_dft = tf.signal.fftshift(dft)
    magnitude_spectrum = tf.abs(shifted_dft)
    log_magnitude = tf.math.log(magnitude_spectrum + 1e-8)  # Adding a small value to avoid log(0)
    img = 20 * log_magnitude
    return img, label

def process_grays(image, label):
  img = tf.image.rgb_to_grayscale(image)
  img = tf.squeeze(img, axis=0)
  return img, label

Dividir o dataset de treino em treino e validação.

In [ ]:
trainset, valset = split_train_val(0.8,dataset)

length: 52380
Train size 41904
Train tf.Tensor(41904, shape=(), dtype=int64)
Val tf.Tensor(10476, shape=(), dtype=int64)


### No Filter

In [ ]:
dataV3 = trainset
MODEL_NAME = "None"

### All

In [ ]:
dataV3 = trainset

dataV3 = trainset.map(process_brightness)
dataV3 = dataV3.concatenate(trainset.map(process_contrast))
dataV3 = dataV3.concatenate(trainset.map(process_hue))
dataV3 = dataV3.concatenate(trainset.map(process_saturation))
dataV3 = dataV3.concatenate(trainset.map(process_rotate))
dataV3 = dataV3.concatenate(trainset.map(process_shear))
dataV3 = dataV3.concatenate(trainset.map(process_translate))
dataV3 = dataV3.concatenate(trainset.map(process_crop))
dataV3 = dataV3.concatenate(trainset.map(process_edges))

MODEL_NAME = "All"

## Colors

In [ ]:
# color ops

dataColors = trainset.map(process_brightness)
dataColors = dataColors.concatenate(trainset.map(process_contrast))
dataColors = dataColors.concatenate(trainset.map(process_hue))
dataColors = dataColors.concatenate(trainset.map(process_saturation))

dataV3 = dataColors

MODEL_NAME = "Colors_Concat"

## Position



In [ ]:
dataPosition = trainset.map(process_rotate)
dataPosition = dataPosition.concatenate(trainset.map(process_shear))
dataPosition = dataPosition.concatenate(trainset.map(process_translate))
dataPosition = dataPosition.concatenate(trainset.map(process_crop))

dataV3 = dataPosition

MODEL_NAME = "Position_Concat"

## Brigthness

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_brightness)


MODEL_NAME = "Brightness"

## Contrast

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_contrast)


MODEL_NAME = "Contrast"

## Hue

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_hue)

MODEL_NAME = "Hue"

## Saturation


In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_saturation)

MODEL_NAME = "Saturation"


## Rotate

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_rotate)

MODEL_NAME = "Rotate"

## Shear

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_shear)

MODEL_NAME = "Shear"

###Test
395/395 - 5s - loss: 92.5414 - accuracy: 0.8739 - 5s/epoch - 13ms/step
###Validation
246/246 - 16s - loss: 0.0075 - accuracy: 0.9976 - 16s/epoch - 66ms/step
###Train
980/980 - 5s - loss: 0.0074 - accuracy: 0.9980 - 5s/epoch - 5ms/step


## Translate

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_translate)
MODEL_NAME = "Translate"

## Crop

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_crop)
MODEL_NAME = "Crop"

## Sobel

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_edges)
MODEL_NAME = "Sobel"

## Grays

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_grays)
MODEL_NAME = "Grays"

## Fourier

In [ ]:
dataV3 = trainset
# color ops
dataV3 = dataV3.map(process_fourier)
MODEL_NAME = "Fourier"

## Batch

In [ ]:
BATCH_SIZE = 32

dataV3 = dataV3.cache()
dataV3 = dataV3.shuffle(buffer_size = 81600)
dataV3 = dataV3.batch(batch_size = BATCH_SIZE)
dataV3 = dataV3.prefetch(buffer_size = AUTOTUNE)

valset = valset.cache()
valset = valset.shuffle(buffer_size = 81600)
valset = valset.batch(batch_size = BATCH_SIZE)
valset = valset.prefetch(buffer_size = AUTOTUNE)

## Model

Mostrar as imagens de uma batch

In [ ]:
image_batch, label_batch = next(iter(dataV3))
show_batch(image_batch, label_batch.numpy())

Definição do modelo

In [ ]:
def model_III(classCount, imgSize, channels):

    model = Sequential()

    model.add(Conv2D(128, (5, 5),
                     input_shape=(imgSize, imgSize, channels)
                     ))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2D(128, (5, 5) ))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (5, 5) ) )
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.2))

    model.add(Dense(classCount, activation='softmax'))


    opt = Adam(lr=0.0001)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=[ 'accuracy'])
    return model

In [ ]:
modelV3 = model_III(43, 32, 3)
log_and_model_path= 'drive/MyDrive/models'

file_pathV3 = f'{log_and_model_path}/{MODEL_NAME}/cp.ckpt'

Treino do modelo

In [ ]:
callbacksV3 = prepare_callbacks(file_pathV3)

historyV3 = modelV3.fit(dataV3,
          epochs = 50,
          validation_data = valset,
          callbacks = callbacksV3)

Resultados do modelo treinado

In [ ]:
modelV3.load_weights(file_pathV3)
evalV3 = modelV3.evaluate(testset, verbose=2)

Resultados dos modelos de uma pasta

In [ ]:
folder = 'drive/MyDrive/models'
for file in os.listdir(folder):
  file_pathV3 = f'{folder}/{file}/cp.ckpt'
  modelV3.load_weights(file_pathV3)
  evalV3 = modelV3.evaluate(testset, verbose=2)

# Voting Ensemble

In [ ]:
NUM_CLASSES = 43
TRAIN_ONLINE = False

Criação do modelo igual ao modelo acima com apenas diferença na separação dos logits


In [ ]:
def create_model(classCount, imgSize, channels):

    modelLogits = Sequential()

    modelLogits.add(Conv2D(128, (5, 5),
                     input_shape=(imgSize, imgSize, channels)
                     ))
    modelLogits.add(BatchNormalization())
    modelLogits.add(LeakyReLU(alpha=0.01))

    modelLogits.add(Conv2D(128, (5, 5) ))
    modelLogits.add(BatchNormalization())
    modelLogits.add(LeakyReLU(alpha=0.01))
    modelLogits.add(MaxPooling2D(pool_size=(2, 2)))

    modelLogits.add(Conv2D(256, (5, 5) ) )
    modelLogits.add(BatchNormalization())
    modelLogits.add(LeakyReLU(alpha=0.01))
    modelLogits.add(MaxPooling2D(pool_size=(2, 2)))

    modelLogits.add(Flatten())
    modelLogits.add(Dense(128))
    modelLogits.add(LeakyReLU(alpha=0.01))
    modelLogits.add(Dropout(0.2))

    modelLogits.add(Dense(classCount))

    output = Activation('softmax')(modelLogits.output)

    model = tf.keras.Model(modelLogits.inputs, output)

    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=[ 'accuracy'])
    return model, modelLogits


In [ ]:
WEIGHT_FILE_PREFIX = 'drive/MyDrive/models_balanced'
file_path_prefix = f'{WEIGHT_FILE_PREFIX}'


import os
def train_models(train, val,file_path_prefix):
    models = []
    models_paths = os.listdir(WEIGHT_FILE_PREFIX)
    models_paths = [ x for x in models_paths if not x.endswith('Concat')]
    NUM_MODELS = len(models_paths)
    histories = []
    for i in range(NUM_MODELS):
        model, modelL = create_model(NUM_CLASSES,IMAGE_SIZE,3)
        if TRAIN_ONLINE:
            callbacks = prepare_callbacks(f'{file_path_prefix}_{i:02}/cp.ckpt')

            hist = model.fit(train,
                            epochs=50,
                            validation_data = valset,
                            callbacks = callbacks)

        models.append([model, modelL])

    return models

In [ ]:
if TRAIN_ONLINE:
  models_V3= train_models(dataV3, valset,file_path_prefix)
else:
  models_V3 = train_models(None, None,file_path_prefix)
NUM_MODELS = (len(models_V3))
print(models_V3)

In [ ]:
def load_weights(models, file_path_prefix):

    models_paths = os.listdir(WEIGHT_FILE_PREFIX)
    models_paths = [ x for x in models_paths if not x.startswith('.')]

    for i in range(NUM_MODELS):

        file_path = f'{file_path_prefix}/{models_paths[i]}/cp.ckpt'
        print(i,models_paths[i])

        models[i][0].load_weights(file_path)
        models[i][0].save('temp.hdf5')
        models[i][1].load_weights('temp.hdf5', by_name=True)

file_path_prefix = 'drive/MyDrive/models_balanced'
load_weights(models_V3, file_path_prefix)


In [ ]:
def evaluate_models(models):

    accuracy = 0

    for i in range(NUM_MODELS):

        eval = models[i][0].evaluate(testset, verbose = 2)
        accuracy += eval[1]

    print(f'average accuracy: {(accuracy/NUM_MODELS)*100:.3f}')


evaluate_models(models_V3)

In [ ]:
def get_labels_logits_and_preds(models):

    preds = [[] for _ in range(NUM_MODELS) ]
    logits = [[] for _ in range(NUM_MODELS)]
    labels = []
    for images, labs in testset.take(-1):

        labels.extend(labs.numpy())
        for i in range(NUM_MODELS):

            preds[i].extend(models[i][0].predict(images))
            logits[i].extend(models[i][1].predict(images))

    labels = [np.argmax(i) for i in labels]

    return labels, logits, preds

#labels_V1, logits_V1, preds_V1 = get_labels_logits_and_preds(models_V3)

In [ ]:
def get_class_preds(preds):

    class_preds = []

    for i in range(test_dataset_length):

        c = []
        for m in range(NUM_MODELS):

            c.append(np.argmax(preds[m][i]))
        class_preds.append(c)

    return class_preds


#class_preds_V1 = get_class_preds(preds_V1)

In [ ]:
def get_class_from_sum_of_logits(logits):

    sum_logits = []

    for i in range(test_dataset_length):

        log = logits[0][i]
        for m in range(1, NUM_MODELS):
            log = np.add(log, logits[m][i])
        sum_logits.append(np.argmax(log))
    return(sum_logits)

#class_logits_V1 = get_class_from_sum_of_logits(logits_V1)

In [ ]:
import collections
def get_stats(labels, class_preds, class_logits):

    all_correct = 0
    all_incorrect = 0
    maj_vote = 0
    maj_wrong = 0
    tie = 0
    count = 0
    log_ok = 0
    log_ko = 0

    for k in range(test_dataset_length):

        counter = collections.Counter(class_preds[k])
        if len(counter) == 1:
            if counter.most_common(1)[0][0] == labels[k]:
                all_correct += 1
            else:
                all_incorrect += 1
        else:
            aux = counter.most_common(2)
            if aux[0][1] > aux[1][1] and aux[0][0] == labels[k]:
                maj_vote += 1
            if aux[0][1] > aux[1][1] and aux[0][0] != labels[k]:
                maj_wrong += 1
            elif aux[0][1] == aux[1][1]:
                tie += 1
        if class_logits[k] == labels[k]:
            log_ok += 1
        else:
            log_ko += 1
        count += 1

    return [count, all_correct, all_incorrect, maj_vote, tie, maj_wrong, log_ok, log_ko]

#res = get_stats(labels_V1, class_preds_V1, class_logits_V1)
#print(res, res[6]/res[0])

In [ ]:
labels_V3, logits_V3, preds_V3 = get_labels_logits_and_preds(models_V3)
class_preds_V3 = get_class_preds(preds_V3)
class_logits_V3 = get_class_from_sum_of_logits(logits_V3)

res = get_stats(labels_V3, class_preds_V3, class_logits_V3)
print(res, res[6]/res[0])

1/1 [==============================] - 0s 93ms/step


1/1 [==============================] - 0s 97ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 137ms/step
[12630, 12023, 4, 523, 20, 60, 12561, 69] 0.9945368171021378


# Stacked Ensemble

In [ ]:
## Build list of inputs and labels for test class

test_logits_preds = []

for i in range(test_dataset_length):

    aux = []
    for m in range(NUM_MODELS):
        aux.extend(logits_V3[m][i])

    test_logits_preds.append(aux)


## Compute logit predictions and training labels



In [ ]:
logits_train = [[] for _ in range(NUM_MODELS)]
labels_aux = []
for images, labs in dataV3.take(-1):

    labels_aux.extend(labs.numpy())
    for i in range(NUM_MODELS):

        logits_train[i].extend(models_V3[i][1].predict(images))

labels_train = [np.argmax(i) for i in labels_aux]

## Build list of train inputs


In [ ]:
print(len(labels_train))

In [ ]:

train_logits_preds = []

for i in range(len(labels_train)):

    aux = []

    for m in range(NUM_MODELS):

        aux.extend(logits_train[m][i])

    train_logits_preds.append(aux)

In [ ]:
print(len(labels_train), len(train_logits_preds))

In [ ]:
stack_model  = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(len(train_logits_preds[0]),)),

  tf.keras.layers.Dense(256),
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128),
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(64),
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),

  tf.keras.layers.Dense(8, activation='softmax')
])


In [ ]:
stack_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
print(len(labels_train), len(train_logits_preds), len(logits_train[0]))

In [ ]:
stack_model.fit(np.asarray(train_logits_preds),np.asarray(labels_train),epochs=100, batch_size=32,
                        validation_data = (np.asarray(test_logits_preds), np.asarray(labels_V3)))

In [ ]:
pred = stack_model.predict(np.asarray(test_logits_preds))

In [ ]:
correct = 0

for i in range(test_dataset_length):
    if np.argmax(pred[i]) == labels_V3[i] :
        correct += 1

print(correct, correct/4170)